# Tutorial 1: Modeling and Basic Importance Sampling Inference

This tutorial shows how to write a probabilistic model in Gen, and how to implement a simple inference algorithm for this model.

Specifically, will implement an inference algorithm that infers the probable destination of an autonomous agent from its observed motion in a two-dimensional environment with obstacles. The algorithm will employ a generative model that includes (i) a prior distribution on the destination of the agent, and (ii) an algorithmic model for how the agent plans its movement based on its destination, (iii) a statistical model of the noise in our measurements of the agent's motion. This class of model draws inspiration from models of human cognition from the computational cognitive science literature [1]. We use this model because the results of inference are intuitive from everyday life and are straightforward to visualize.

We will break down this modeling and inference tasks into the following steps:

- **Step 1:** Implement geometric primitives for a two-dimensional scene that contains obstacles.

- **Step 2:** Implement a planning algorithm that takes the two-dimensional scene, and a starting point within the scene, and a destination point within the scene, and generates an efficient (short) path from the starting point to the destination point.

- **Step 3:** Write a generative function for the observed motion of of an autonomous agent, using the Julia code written in Steps 1 and 2.

- **Step 4:** Explore the the assumptions, or prior beliefs, that our inferences will be based on, by sampling traces of the generative function.

- **Step 5:** Use a simple importance sampling algorithm to sample probable destination points for the agent, given an example data set of observed motion.

- **Step 6:** Peek under the hood of the inference algorithm to build some intuition for how it works. 

[1] Baker, Chris L., Rebecca Saxe, and Joshua B. Tenenbaum. "Action understanding as inverse planning." Cognition 113.3 (2009): 329-349.

### The Gen package

First, we load the Gen package, which brings into scope the built-in Gen modeling languages and inference library methods. We won't use Gen itself until Step 3, but we load it here because packages are usually loaded at the beginning of a script.

In [ ]:
using Gen

## Step 1: Implement geometric primitives for a two-dimensional scene with obstacles.

This step is implemented using regular Julia code.

First, we want a data type for a location in a two-dimensional plane:

In [ ]:
struct Point
    x::Float64
    y::Float64
end

We will need to measure the distance between two points, so we define a method for this:

In [ ]:
function dist(a::Point, b::Point)
    dx = a.x - b.x
    dy = a.y - b.y
    sqrt(dx * dx + dy * dy)
end

We will also need to test whether two line segments intersect. The method below tests whether the line segment between the points `a1` and `a2` intersects the segment between the points `b1` and `b2`. The implementation of this primitive is based upon https://algs4.cs.princeton.edu/91primitives/. 

In [ ]:
function line_segments_intersect(a1::Point, a2::Point, b1::Point, b2::Point)
    if ccw(a1, a2, b1) * ccw(a1, a2, b2) > 0
        false
    elseif ccw(b1, b2, a1) * ccw(b1, b2, a2) > 0
        false
    else
        true
    end
end

ccw(a::Point, b::Point, c::Point) = (b.x - a.x) * (c.y - a.y) - (c.x - a.x) * (b.y - a.y)

Next, we define a data type to represent obstacles in the scene. An obstacle will be represented by a polygon. We also define a method to test whether a given line defined by points `a1` and `a2` intersects the obstacle:

In [ ]:
struct Obstacle
    vertices::Vector{Point}
end

function obstacle_intersects_line_segment(obstacle::Obstacle, a1::Point,  a2::Point)
    n = length(obstacle.vertices)
    for start_vertex_idx=1:n
        end_vertex_idx = start_vertex_idx % n + 1 # loop over to 1
        v1 = obstacle.vertices[start_vertex_idx]
        v2 = obstacle.vertices[end_vertex_idx]
        if line_segments_intersect(v1, v2, a1, a2)
            return true
        end
    end
    false
end

Now, we define a data type to represent the two-dimensional scene. The scene spans a rectangle of on the two-dimensional x-y plane, and contains a list of obstacles. Each obstacle is a polygon defined by a list of vertex points. We also define a method to compute whether a given line is obstructed by any obstacles in the scene.

In [ ]:
struct Scene
    xmin::Float64
    xmax::Float64
    ymin::Float64
    ymax::Float64
    obstacles::Vector{Obstacle}
end

Scene(xmin, xmax, ymin, ymax) = Scene(xmin, xmax, ymin, ymax, Obstacle[])

add_obstacle!(scene, obstacle::Obstacle) = push!(scene.obstacles, obstacle)

function line_is_obstructed(scene::Scene, a1::Point, a2::Point)
    for obstacle in scene.obstacles
        if obstacle_intersects_line_segment(obstacle, a1, a2)
            return false
        end
    end
    true
end

Finally, we write some methods that allow us to concisely construct walls (line-shaped obstacles that are either vertically or horizontally oriented), and square-shaped obstacles (representing trees).

In [ ]:
function make_wall(vertical::Bool, start::Point, length::Float64, thickness::Float64)
    vertices = Vector{Point}(undef, 4)
    vertices[1] = start
    dx = vertical ? thickness : length
    dy = vertical ? length : thickness
    vertices[2] = Point(start.x + dx, start.y)
    vertices[3] = Point(start.x + dx, start.y + dy) 
    vertices[4] = Point(start.x, start.y + dy)
    Obstacle(vertices)
end 

function make_tree(center::Point, size::Float64)
    vertices = Vector{Point}(undef, 4)
    vertices[1] = Point(center.x - size/2, center.y - size/2)
    vertices[2] = Point(center.x + size/2, center.y - size/2)
    vertices[3] = Point(center.x + size/2, center.y + size/2)
    vertices[4] = Point(center.x - size/2, center.y + size/2)
    Obstacle(vertices)
end

## Step 2: Implement a planning algorithm

In this step, we implement a simple planning algorithm based on the rapidly exploring random tree (RRT) algorithm [2]. The planning algorithm will take as input (i) the scene, (ii) the start point, and (iii) the destination point, and will produce a sequence of points that starts with the start point and ends with the destination point, such the line of sight between each consecutive point does not intersect any obstacles, or return failure if no path could be found.

[2] Rapidly-exploring random trees: A new tool for path planning. S. M. LaValle. TR 98-11, Computer Science Dept., Iowa State University, October 1998,

First, we define the data types for the RRT. The tree is a list of nodes, each of which stores the point (`conf`), a reference to the parent node, the vector from the parent to the node (`control`), and the total distance from the start node to this node, following the edges of the tree (`dist_from_start`).

In [ ]:
struct RRTNode
    conf::Point
    parent::Union{Nothing,RRTNode}
    control::Union{Nothing,Point}
    dist_from_start::Float64
end

The RRT is a list of nodes, where the first node is the 'root' node:

In [ ]:
struct RRT
    nodes::Vector{RRTNode}
end

function RRT(root_conf::Point)
    nodes = RRTNode[RRTNode(root_conf, nothing, nothing, 0.)]
    RRT(nodes)
end

add_node!(tree::RRT, node::RRTNode) = push!(tree.nodes, node)

root(tree::RRT) = tree.nodes[1]

One of the key operations used in the RRT algorithm is generating a random point in the scene:

In [ ]:
function random_point_in_scene(scene::Scene)
    x = rand() * (scene.xmax - scene.xmin) + scene.xmin
    y = rand() * (scene.ymax - scene.ymin) + scene.ymin
    Point(x, y)
end

Another key operation is finding the point on the tree that is closest to some other point:

In [ ]:
function nearest_node_on_tree(tree::RRT, conf::Point)
    nearest::RRTNode = root(tree)
    best_dist::Float64 = dist(nearest.conf, conf)
    for node in tree.nodes
        d = dist(node.conf, conf)
        if d < best_dist
            best_dist = d 
            nearest = node
        end
    end 
    nearest
end

Finally, we implement the RRT algorithm, which generates a RRT. It operates by randomly picking points from the scene and trying to connect them to the tree with a non-obstructed line-of-site.

In [ ]:
function generate_rrt(scene::Scene, init::Point, iters::Int, dt::Float64)
    tree = RRT(init) # init is the root of tree
    for iter=1:iters
        rand_conf::Point = random_point_in_scene(scene)
        near_node::RRTNode = nearest_node_on_tree(tree, rand_conf)
        
        dist_to_target = dist(near_node.conf, rand_conf)
        diff = Point(rand_conf.x - near_node.conf.x, rand_conf.y - near_node.conf.y)
        distance_to_move = min(dt, dist_to_target)
        scale = distance_to_move / dist_to_target
        control = Point(scale * diff.x, scale * diff.y)

        # go in the direction of target_conf from start_conf 
        new_conf = Point(near_node.conf.x + control.x, near_node.conf.y + control.y)

        # test the obstacles
        failed = line_is_obstructed(scene, near_node.conf, new_conf)
        
        if !failed
            dist_from_start = near_node.dist_from_start + distance_to_move
            new_node = RRTNode(new_conf, near_node, control, dist_from_start)
            add_node!(tree, new_node)
        end
    end
    tree
end

Given a RRT, and a destination point, we can find a path from the root of the RRT to the destination point, by finding a node on the tree that has a clear line-of-sight to the destination node, and is also as close as possible to the destination node. If there is no node on the tree with a clear line-of-sight to the destination, we return the value `nothing`:

In [ ]:
function find_tree_route_to_dest(tree::RRT, dest::Point)
    best_node = tree.nodes[1]
    min_cost = Inf
    path_found = false
    for node in tree.nodes
        # check for line-of-site to the destination
        clear_path = !line_is_obstructed(scene, node.conf, dest)
        cost = node.dist_from_start + (clear_path ? dist(node.conf, dest) : Inf)
        if cost < min_cost
            path_found = true
            best_node = node
            min_cost = cost
        end
    end
    if path_found
        (best_node, min_cost)
    else
        (nothing, min_cost)
    end
end

Then, we obtain a sequence of points that define the path by walking back along the tree from the best tree  found node, back to the start node:

In [ ]:
struct Path
    points::Vector{Point}
end

function walk_tree_to_root(node::RRTNode, start::Point, dest::Point)
    points = Point[dest]
    while node.parent != nothing
        push!(points, node.conf)
        node = node.parent
    end
    push!(points, start)
    @assert points[end] == start # the start point
    @assert points[1] == dest
    Path(reverse(points))
end

The paths along the tree that are generated by the RRT algorithm are generally not very direct. We want our agent to take fairly direct paths from its starting location to the destination. Therefore, we use the following path-refinement procedure to optimize the path points to shorten the length of the path while still avoiding obstruction by obstacles.

In [ ]:
function refine_path(scene::Scene, original::Path, iters::Int, std::Float64)
    # do stochastic optimization
    new_points = copy(original.points)
    num_interior_points = length(original.points) -2
    if num_interior_points == 0
        return original
    end
    for i=1:iters
        point_idx = 2 + (i % num_interior_points)
        @assert point_idx > 1 # not start
        @assert point_idx < length(original.points) # not dest
        prev_point = new_points[point_idx-1]
        point = new_points[point_idx]
        next_point = new_points[point_idx+1]
        adjusted = Point(point.x + randn() * std, point.y + randn() * std)
        cur_dist = dist(prev_point, point) + dist(point, next_point)
        ok_backward = !line_is_obstructed(scene, prev_point, adjusted)
        ok_forward = !line_is_obstructed(scene, adjusted, next_point)
        if ok_backward && ok_forward
            new_dist = dist(prev_point, adjusted) + dist(adjusted, next_point)
            if new_dist < cur_dist
                # accept the change
                new_points[point_idx] = adjusted
            end
        end
    end
    Path(new_points)
end

Finally, we combine each of the steps we just defined into a path-planning function. If a path could not be found, we return the value `nothing`. Otherwise, we return a `Path` value The path-planning function has parameters for how to grow the RRT (`rrt_iters` and `rrt_dt`) and how to perform the refinement (`refine_iters` and `refine_std`).

In [ ]:
struct PlannerParams
    rrt_iters::Int
    rrt_dt::Float64 # the maximum proposal distance
    refine_iters::Int
    refine_std::Float64
end

function plan_path(start::Point, dest::Point, scene::Scene, params::PlannerParams)
    
    # Generate a rapidly exploring random tree
    tree = generate_rrt(scene, start, params.rrt_iters, params.rrt_dt)

    # Find a route from the root of the tree to a node on the tree that has a line-of-sight to the destination
    (maybe_node, min_cost) = find_tree_route_to_dest(tree, dest)
    
    if maybe_node == nothing
        
        # No route found
        nothing
    else
        
        # Route found
        node::RRTNode = something(maybe_node)
        path = walk_tree_to_root(node, start, dest)
        refine_path(scene, path, params.refine_iters, params.refine_std)
    end
end



We will assume that the agent moves along its path a constant speed. The cell below defines a method that computes the locations of the agent at a set of timepoints, given the path and the speed of the agent.

In [ ]:
function compute_distances_from_start(path::Path)
    distances_from_start = Vector{Float64}(undef, length(path.points))
    distances_from_start[1] = 0.0
    for i=2:length(path.points)
        distances_from_start[i] = distances_from_start[i-1] + dist(path.points[i-1], path.points[i])
    end
    return distances_from_start
end

function walk_path(path::Path, speed::Float64, times::Array{Float64,1})
    distances_from_start = compute_distances_from_start(path)
    locations = Vector{Point}(undef, length(times))
    locations[1] = path.points[1]
    for (time_idx, t) in enumerate(times)
        if t < 0.0
            error("times must be positive")
        end
        desired_distance = t * speed
        used_up_time = false
        # NOTE: can be improved (iterate through path points along with times)
        for i=2:length(path.points)
            prev = path.points[i-1]
            cur = path.points[i]
            dist_to_prev = dist(prev, cur)
            if distances_from_start[i] >= desired_distance
                # we overshot, the location is between i-1 and i
                overshoot = distances_from_start[i] - desired_distance
                @assert overshoot <= dist_to_prev
                past_prev = dist_to_prev - overshoot
                frac = past_prev / dist_to_prev
                locations[time_idx] = Point(prev.x * (1. - frac) + cur.x * frac,
                                     prev.y * (1. - frac) + cur.y * frac)
                used_up_time = true
                break
            end
        end
        if !used_up_time
            # sit at the goal indefinitely
            locations[time_idx] = path.points[end]
        end
    end
    locations
end

## Step 3: Write a generative function for the motion of an autonomous agent

We now can write a generative function that models the behavior of an autonomous agent.

In [ ]:
@gen function model(scene::Scene, measurement_times::Vector{Float64})

    # sample the start point of the agent from the prior
    start_x = @addr(uniform(0, 1), :start_x)
    start_y = @addr(uniform(0, 1), :start_y)
    start = Point(start_x, start_y)

    # sample the destination point of the agent from the prior
    dest_x = @addr(uniform(0, 1), :dest_x)
    dest_y = @addr(uniform(0, 1), :dest_y)
    dest = Point(dest_x, dest_y)

    # plan a path that avoids obstacles in the scene
    maybe_path::Union{Nothing,Path} = plan_path(start, dest, scene, PlannerParams(300, 3.0, 2000, 1.))
    
    # sample the speed from the prior
    speed = @addr(uniform(0, 1), :speed)

    if maybe_path == nothing
        
        # path planning failed, assume the agent stays as the start location indefinitely
        locations = fill(start, length(measurement_times))
    else
        
        # path planning succeeded, move along the path at constant speed
        path = something(maybe_path)
        locations = walk_path(path, speed, measurement_times)
    end

    # generate noisy measurements
    noise = 0.1 * @addr(uniform(0, 1), :noise)
    for (i, point) in enumerate(locations)
        @addr(normal(point.x, noise), i => :x)
        @addr(normal(point.y, noise), i => :y)
    end

    (start, dest, speed, noise, maybe_path, locations)
end

In [ ]:
scene = Scene(0, 1, 0, 1)
add_obstacle!(scene, make_tree(Point(0.30, 0.20), 0.1))
add_obstacle!(scene, make_tree(Point(0.83, 0.80), 0.1))
add_obstacle!(scene, make_tree(Point(0.80, 0.40), 0.1))
horizontal = false
vertical = true
wall_thickness = 0.02
add_obstacle!(scene, make_wall(horizontal, Point(0.20, 0.40), 0.40, wall_thickness))
add_obstacle!(scene, make_wall(vertical, Point(0.60, 0.40), 0.40, wall_thickness))
add_obstacle!(scene, make_wall(horizontal, Point(0.60 - 0.15, 0.80), 0.15 + wall_thickness, wall_thickness))
add_obstacle!(scene, make_wall(horizontal, Point(0.20, 0.80), 0.15, wall_thickness))
add_obstacle!(scene, make_wall(vertical, Point(0.20, 0.40), 0.40, wall_thickness))

In [ ]:
(trace, _) = initialize(model, (scene, [1.0, 2.0]))